# MUSIC model predictions

## Imports and Constants

In [1]:
import math
import os
from itertools import combinations

import numpy as np
from scipy.io import wavfile
from scipy import signal
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rc
from pandas.plotting import register_matplotlib_converters

import pyroomacoustics as pra
from pyroomacoustics.utilities import normalize
from pyroomacoustics.transform import stft
from sklearn.metrics import confusion_matrix, accuracy_score

# Label resolution of angles
RESOLUTION = 10

# Determines the overlap of samples between consecutive features
STEP = 1024

# Training rooms dimensions
ROOMS = {
    'small' : np.array([4, 4, 3]),
    'medium' : np.array([6, 6, 3]),
    'large' : np.array([8, 8, 3])
}

# Testing rooms dimensions
TEST_ROOMS = {
    'small' : np.array([5, 5, 2]),
    'medium' : np.array([7, 7, 2]),
    'large' : np.array([9, 9, 2])
}

class Path:
    def __init__(self, name, path, max_angle):
        self.name = name
        self.path = path
        self.max_angle = max_angle

AUDIO_PATH = [
    Path('full', '..\\data\\full_circle\\0.001', 360),
    Path('half', '..\\data\\half_circle\\0.001', 180)
]

GENERATE_DATA_FOR_COMPARISON = False

## Utils

In [2]:
def compute_stft_matrix(signal, frame_size=256):    
    # Default value for overlap
    overlap = frame_size // 2
    
    # Calculate Short-time Fourier transform and return
    observation = stft.analysis(signal, L=frame_size, hop=overlap)

    # Transpose from [#freq_bins,#frames,#channels] to [#channels,#frames,#freq_bins]
    # [250, 129, 2] -> [2, 129, 250]
    return np.transpose(observation, axes=[2, 1, 0])



def get_music_prediction(X, max_angle, room_size='small', resolution=RESOLUTION, fs=16000, frame_size=256):    
    # Get microphone locations
    w = ROOMS[room_size][0]
    l = ROOMS[room_size][1]
    h = ROOMS[room_size][2]
    mic_pos=np.c_[[w/2+0.1, 0, 0],[w/2-0.1, 0, 0]] 

    # Run MUSIC (MUltiple SIgnal Classication) algorithm for DOA (direction-of-arrival)
    doa = pra.doa.MUSIC(mic_pos, fs, nfft=frame_size, n_grid=(360 // resolution), num_src=1)
    doa.locate_sources(X)
    pred = doa.azimuth_recon[0] 

    # Account for angles from 0 to 180 only
    if max_angle == 180:
        if pred > np.pi:
            pred = (2 * np.pi - pred) * 180 / np.pi
        else:
            pred =  pred * 180 / np.pi

    if max_angle == 360:
        spatial_resp = doa.grid.values
        min_val = spatial_resp.min()
        max_val = spatial_resp.max()
        spatial_resp = (spatial_resp - min_val) / (max_val - min_val)
        phi_plt = doa.grid.azimuth / 2
        c_phi_plt = np.r_[phi_plt, phi_plt[0]]
        c_dirty_img = np.r_[spatial_resp, spatial_resp[0]]
          
        # Find average prediction\n",
        pred = np.argpartition(c_dirty_img* 10. + 1, -1)[-1:]
        pred_angle = c_phi_plt[pred] * 360 / np.pi
        pred = round(round(np.average(pred_angle)))

    return round(pred)



def get_all_predictions(resolution=RESOLUTION, audio_path=AUDIO_PATH[1]):    
    y_true = []
    y_pred = []
    info = []
    
    files = [file for file in os.listdir(audio_path.path)] # if 'test' in file]

    # Loop through all WAVs
    for i, file in enumerate(files):
        if file[-3:] != 'wav': 
            continue

        path = os.path.join(audio_path.path, file)
        fs, wav_signals = wavfile.read(path)
        label = int(file.split('_')[2])
        
        # Get the file information (distance, room type, sound signal angle)
        dist = int(file.split('_')[4])
        room = file.split('_')[6]
        y = [(dist, room, label)]
        
        # Convert the wav signal into frequency domain
        X = compute_stft_matrix(wav_signals)
        pred = get_music_prediction(X, audio_path.max_angle, room_size=room, resolution=resolution)

        # Store actual and predicted labels
        y_true.extend(y)
        y_pred.append(pred) 
        print(f'File {i+1}/{len(files)}', end='\r')
       
    y_true = np.array(y_true)
    
    info = y_true[:, :-1]
    y_true = list(y_true[:, -1].astype(int))
    y_pred = list(np.array(y_pred).astype(int))
    return y_true, y_pred, info

In [3]:
y_true, y_pred, info = get_all_predictions(resolution=RESOLUTION, audio_path=AUDIO_PATH[1])

## Visualization

In [4]:
# Create .csv dataset used inside Comparison.ipynb
# Apply for a single file simulations 
if GENERATE_DATA_FOR_COMPARISON:
    df = pd.DataFrame(columns=['y_true', 'y_pred'])
    for i in range(len(y_true)):
        if y_true[i] in df['y_true'].values:
            get_current_value = df.loc[df['y_true'] == y_true[i], 'y_pred'].values
            df.loc[df['y_true'] == y_true[i], 'y_pred'] = np.average([get_current_value[0], y_pred[i]])
        else: 
            df.loc[len(df)] = [y_true[i], y_pred[i]]
    df = df.sort_values(by=['y_true'])
    df.to_csv('../comparison/MUSIC_data.csv', index=False)

In [6]:
def plot_cm(y_true, y_pred, class_names, font_scale=0.8):
    # Plot the confusion matrix, use the class_names as the labels
    # only if the 180 degree scenario is used

    cm = confusion_matrix(y_true, y_pred, normalize='true')    
    fig, ax = plt.subplots(figsize=(24, 20)) 

    cmap = sns.diverging_palette(220, 20, sep=20, as_cmap=True)
    sns.heatmap(cm, cmap=cmap, annot=False, linewidth=0.01, ax=ax)

    plt.ylabel('Actual', fontsize=40)
    plt.xlabel('Predicted', fontsize=40)
    #ax.set_xticklabels(class_names, fontsize=24)
    #ax.set_yticklabels(class_names, rotation=0, fontsize=24)
    # use matplotlib.colorbar.Colorbar object
    cbar = ax.collections[0].colorbar
    cbar.ax.tick_params(labelsize=40)
    plt.show()
    
    
def rmse(y_true, y_pred):
    # Calculate the root mean square error

    y_true, y_pred = np.array(y_true), np.array(y_pred)
    total = sum(min(abs(y_t - y_p), (180 - abs(y_t - y_p))) ** 2 for y_t, y_p in zip(y_true, y_pred))
    rms = ((total / len(y_true)) ** 0.5)
    
    if type(rms) == np.ndarray:
        rms = rms[0]
        
    return round(rms, 3)

In [ ]:
# Plot confusion matrix and report accuracy
plot_cm(y_true, y_pred, np.unique(y_true))
accuracy = accuracy_score(y_true, y_pred)

print(f'Accuracy: {round(accuracy, 6)}')
print(f'RMSE: {rmse(y_true, y_pred)}')

In [ ]:
def get_entries_with_property(info, prop, value):
    if prop == 'distance': i = 0
    elif prop == 'room': i = 1
        
    info = info[:, i]
    return np.where(info == value)

# Evaluate performance for different properties
print('Room sizes')
for room in ROOMS:
    indices = get_entries_with_property(info, 'room', room)
    y_true_room, y_pred_room = np.take(y_true, indices)[0], np.take(y_pred, indices)[0]
    accuracy = accuracy_score(y_true_room, y_pred_room)
    print(f"- {room} room accuracy: {round(accuracy, 3)}")
    
print('\nDistances')
for dist in np.unique(info[:, 0]):
    indices = get_entries_with_property(info, 'distance', dist)
    y_true_dist, y_pred_dist = np.take(y_true, indices)[0], np.take(y_pred, indices)[0]
    accuracy = accuracy_score(y_true_dist, y_pred_dist)
    print(f"- {dist} cm distance accuracy: {round(accuracy, 3)}")